## Fourier Neural Operator 1D

In [1]:
## Imports 
import matplotlib.pyplot as plt 
# import numpy as np 
# import scipy.io 
import torch 
import torch.nn as nn

c:\Users\s1612415\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## Set seeds
torch.manual_seed(0)
# np.random.seed(0)

## Create Data

## Build Network

In [3]:
## Get Device for Training
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device.')

Using cpu device.


In [106]:
## Fourier Integral Kernel 1D
class FourierIntegralKernel1D(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, modes: int):
        super(FourierIntegralKernel1D, self).__init__()
        '''
        '''
        self.in_channels = in_channels
        self.out_channels = out_channels 
        self.modes = modes 
        ## Set (random) weights for the linear transform
        weights = torch.rand(self.modes, self.in_channels, self.out_channels, dtype=torch.cfloat) 
        self.weights = nn.Parameter(weights) ## Optional: Scale weights

    def forward(self, v: torch.Tensor) -> torch.Tensor:
        '''
        FFT -> Linear Transform -> Inverse FFT
        '''
        ## FFT
        v_rfft = torch.fft.rfft(v) 

        ## Linear Transform 
        lv_rfft = torch.zeros(v_rfft.shape, dtype=torch.cfloat)
        lv_rfft[:, :self.modes] = torch.einsum('koi, ki -> ko', self.weights, v_rfft[:, :self.modes].permute(1, 0)).permute(1, 0) ## TODO: Check this

        ## Inverse FFT
        v2 = torch.fft.irfft(lv_rfft, n=v.shape[-1])
        return v2
        

In [107]:
## Fourier Network Operator 1D
class FourierNetworkOperator1D(nn.Module):
    def __init__(self, da: int, du: int, modes: int):
        super(FourierNetworkOperator1D, self).__init__()
        '''
        '''
        self.modes = modes

        ## P: Lifts the lower dimensional function to higher dimensional space
        self.P = nn.Conv1d(da, 16, 1) ## TODO: Change da

        ## K: Fourier integral kernel operator
        self.k0 = FourierIntegralKernel1D(16, 16, self.modes)
        self.k1 = FourierIntegralKernel1D(16, 16, self.modes)
        self.k2 = FourierIntegralKernel1D(16, 16, self.modes)
        self.k3 = FourierIntegralKernel1D(16, 16, self.modes)

        ## W: Pointwise linear operator
        self.w0 = nn.Conv1d(16, 16, 1)
        self.w1 = nn.Conv1d(16, 16, 1)
        self.w2 = nn.Conv1d(16, 16, 1)
        self.w3 = nn.Conv1d(16, 16, 1)

        ## Q: Projects the higher dimensional function to lower dimensional space
        self.Q = nn.Conv1d(16, du, 1) ## TODO: Change du

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        '''
        '''
        ## P
        x = self.P(x)

        ## Fourier Layer #0
        ## K
        x1 = self.k0(x)
        ## W
        x2 = self.w0(x)
        ## Sum
        x = x1 + x2
        ## Gelu
        x = nn.functional.gelu(x)

        ## Fourier Layer #1
        ## K
        x1 = self.k1(x)
        ## W
        x2 = self.w1(x)
        ## Sum 
        x = x1 + x2
        ## Gelu
        x = nn.functional.gelu(x)

        ## Fourier Layer #2
        ## K
        x1 = self.k2(x)
        ## W
        x2 = self.w2(x)
        ## Sum
        x = x1 + x2
        ## Gelu
        x = nn.functional.gelu(x)

        ## Fourier Layer #3
        ## K
        x1 = self.k3(x)
        ## W
        x2 = self.w3(x)
        ## Sum
        x = x1 + x2
        ## Gelu
        x = nn.functional.gelu(x)
        
        ## Q
        x = self.Q(x)
        return x


In [108]:
model = FourierNetworkOperator1D(2, 1, 1)
x = torch.tensor([[1, 2, 3, 4, 5],[0.1, 0.11, 0.12, 0.13, 0.14]])
model(x)

tensor([[395.7504, 395.7633, 395.7769, 395.7920, 395.8087]],
       grad_fn=<SqueezeBackward1>)

## Train

## Test